<a href="https://colab.research.google.com/github/Khlebovich-Alexandra/horoscope_generator/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM 
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import itertools
import os
import re
import time
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

TensorFlow 2.x selected.


In [0]:
posts = pd.read_csv('https://raw.githubusercontent.com/Khlebovich-Alexandra/horoscope_generator/master/Data/final_posts.csv', index_col=0)

In [0]:
posts

,text,length,date,type,index in posts,domain,index_before_concat
0,сегодня вы можете почувствовать незащищенность...,339,11 декабря,business,0,ribyhoroscop,0
1,вам звёзды рекомендуют сегодня больше работать...,303,11 декабря,love,0,ribyhoroscop,1
2,"если сегодня вы почувствуете робость, или, тог...",215,11 декабря,simple,0,ribyhoroscop,2
3,сегодня звезды рекомендуют вам заняться коррек...,278,10 декабря,business,1,ribyhoroscop,3
4,"сегодня ваш любимый человек признается в том, ...",273,10 декабря,love,1,ribyhoroscop,4
...,...,...,...,...,...,...,...
26957,"день преобразования космических энергий, получ...",544,2 марта,simple,2648,strelechoroscop,4575
26958,завтра - полон перемен для вас. поэтому строит...,256,1 марта,simple,2649,strelechoroscop,4576
26959,утром не мешкая приступайте к работе или завяз...,441,28 февраля,simple,2650,strelechoroscop,4577
26960,общительность и дружелюбие вас сегодня могут н...,453,27 февраля,simple,2651,strelechoroscop,4578


In [0]:
posts.drop(['date', 'type', 'index in posts', 'domain', 'index_before_concat'], axis=1, inplace=True)

In [0]:
posts.drop(np.argmax(posts.length), inplace=True)

In [0]:
def add_spaces(x):
    literals = ',.?!;:()'
    for literal in literals:
        if literal in x:
            x = x.replace(literal, ' ' + literal + ' ')
            
    pattern_bad_spaces_1 = re.compile(r'(^ +)|( +$)')
    pattern_bad_spaces_2 = re.compile(r'(  +)|(\t)')
    x = re.sub(pattern_bad_spaces_1, '', x)
    x = re.sub(pattern_bad_spaces_2, ' ', x)
    return x

In [0]:
def split(x):
    res = re.split(re.compile(r'[ ]+'), x.lower())
    while '' in res:
        res.remove('')
    return res

In [0]:
def create_vocab(texts):
    texts = pd.Series(texts)
    texts = texts.map(add_spaces)
    texts = texts.map(split)
    word_index = {}
    word_index["<Заполнитель>"] = 0
    word_index["<Начало последовательности>"] = 1
    word_index["<Не используется>"] = 2
    word_index["<Неизвестное слово>"] = 3
    ind = 4
    for text in texts:
        for word in text:
            if not (word in word_index):
                word_index[word] = ind
                ind += 1
    return word_index

In [0]:
def word_to_int(word):
    if word in word_index:
        return word_index[word]
    return 3

In [0]:
def get_word_by_index(index):
    for word, value in word_index.items():
        if index == value:
            return word
    return ''

In [0]:
def create_dataset(text):
    text = add_spaces(text)
    text = split(text)
    text = list(map(word_to_int, text))
    return text

In [0]:
word_index = create_vocab(posts.text)

In [0]:
word_index['.']

14

In [0]:
len(word_index)

47923

In [0]:
print(create_dataset(posts.text[0]))

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 14, 28, 29, 30, 31, 32, 33, 19, 34, 35, 36, 37, 19, 38, 39, 40, 41, 42, 43, 44, 14, 45, 46, 47, 25, 48, 49, 14]


In [0]:
posts.text = posts.text.map(create_dataset)

In [0]:
posts.length = posts.text.map(len)

Save DataFrame and vocabluary

In [0]:
posts.to_csv('posts_word_to_int.csv')

In [0]:
import json

with open('vocab.json', 'w') as fp:
    json.dump(word_index, fp)

Padding seq

In [0]:
def get_list_of_ints(x):
    if isinstance(x, str):
        x = x.strip('][').split(', ')
    return list(map(int, x))

In [0]:
posts = pd.read_csv('posts_word_to_int.csv', index_col=0)
posts.text = posts.text.map(get_list_of_ints)
posts

,text,length
0,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",53
1,"[50, 51, 52, 4, 53, 54, 14, 55, 56, 57, 58, 59...",49
2,"[89, 4, 5, 90, 91, 19, 56, 19, 92, 93, 94, 19,...",40
3,"[4, 109, 52, 50, 110, 111, 56, 112, 113, 114, ...",46
4,"[4, 136, 137, 138, 139, 11, 140, 19, 20, 141, ...",46
...,...,...
26957,"[199, 7429, 7906, 19954, 19, 8101, 20213, 8982...",81
26958,"[3016, 81, 2294, 3598, 47, 25, 14, 302, 6957, ...",40
26959,"[8803, 121, 47918, 13480, 71, 535, 56, 35595, ...",63
26960,"[8673, 9, 5898, 25, 4, 702, 23496, 87, 19815, ...",68


In [0]:
posts.describe()

,length
count,26961.000000
mean,50.237640
std,15.919764
min,4.000000
25%,40.000000
50%,48.000000
75%,58.000000
max,129.000000


In [0]:
maxlen = 50

In [0]:
texts = pad_sequences(posts.text, maxlen, padding='post')
texts = tf.data.Dataset.from_tensor_slices(texts)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = texts.map(split_input_target)

In [0]:
dataset.take(1)

<TakeDataset shapes: ((49,), (49,)), types: (tf.int32, tf.int32)>

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 49), (64, 49)), types: (tf.int32, tf.int32)>

In [0]:
# define model
vocab_size = len(word_index)
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12268288  
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 47923)         49121075  
Total params: 65,327,667
Trainable params: 65,327,667
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
 for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 49, 47923) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([43300, 36774,  3959, 19223, 12408, 17769, 35770, 31458, 18555,
       16962, 37015, 43176, 20071,  6699, 10066, 35625,   267, 21318,
       26535, 39679, 29277, 19953, 11723, 38912, 38630, 36330,  3403,
       44143, 23315, 15201, 14070, 34031, 32983, 43160, 43281, 21253,
        6120, 35009, 17436, 31376, 27662, 27922, 44843, 22289,  4526,
       13606, 29302, 40242, 26342])

Decode these to see the text predicted by this untrained model:

In [0]:
print("Input: \n", repr(" ".join([get_word_by_index(i) for i in input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join([get_word_by_index(i) for i in sampled_indices])))

Input: 
 'вас , кажется , начинается полоса неудач . и с начальником отношения не ладятся , и с новым проектом возникли трудности , которые что-то никак не разрешаются . впрочем , невозможно всё время быть на коне , лучше подойдите к этим проблемам философски и найдите в ситуации положительные стороны'

Next Char Predictions: 
 'доверчиво дребезгом первая опирайтесь недочеты предоставите служения надоедают приготовленное выходящее обжитое бесплатные вестись невзначай неплохого деликатесы главное магических стабильна определённым потратил пробуждения официально мыслят вымещать парить подзаработать напоминающей существу оживление поспать пошатнется исчезло намагничены запретные перессоритесь конкретного песочные поминайте погружены соскучиться собеседникам доходными неверная непрофессиональных помощниками подколам нежданные узкой'


## Train the model

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 49, 47923)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.777466


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=100

In [0]:
history = model.fit(dataset, epochs=EPOCHS)

Train for 421 steps
Epoch 1/100
421/421 [==============================] - 91s 217ms/step - loss: 6.5601
Epoch 2/100
421/421 [==============================] - 90s 214ms/step - loss: 5.8502
Epoch 3/100
421/421 [==============================] - 90s 213ms/step - loss: 5.4948
Epoch 4/100
421/421 [==============================] - 90s 213ms/step - loss: 5.2361
Epoch 5/100
421/421 [==============================] - 90s 213ms/step - loss: 4.9957
Epoch 6/100
421/421 [==============================] - 90s 213ms/step - loss: 4.5184
Epoch 7/100
421/421 [==============================] - 90s 213ms/step - loss: 3.9648
Epoch 8/100
421/421 [==============================] - 90s 213ms/step - loss: 3.3192
Epoch 9/100
421/421 [==============================] - 90s 213ms/step - loss: 2.7647
Epoch 10/100
421/421 [==============================] - 90s 213ms/step - loss: 2.3445
Epoch 11/100
421/421 [==============================] - 90s 213ms/step - loss: 2.0394
Epoch 12/100
421/421 [=====================

In [0]:
model_json = model.to_json()
json_file = open('posts_model_words.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model_words.h5')

## Generate text

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

model.load_weights('posts_model_words.h5')

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            12268288  
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 47923)          49121075  
Total params: 65,327,667
Trainable params: 65,327,667
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, num_of_words, temp):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  text = start_string

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_of_words):
      input_eval = create_dataset(start_string)
      input_eval = pad_sequences([input_eval], maxlen - 1, padding='post')

      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      new_word = get_word_by_index(predicted_id)
      text = text + ' ' + new_word

  return text

In [0]:
generate_text(model, 'сегодня вам ', 10, 4.5)

'сегодня вам  тона посидеть запутано изделия поладите работа <Заполнитель> <Заполнитель> осуществлением способностями'

In [0]:
generate_text(model, 'сегодня вам ', 10, 55)

'сегодня вам  выражаться сидело прозябать обещающий кормления удержитесь перспективах метеор выводя испортятся'

In [0]:
generate_text(model, 'сегодня вам ', 10, 28)

'сегодня вам  возжелать незнакомцам пригорит собственноручно легких расспросы критичный переставьте близкое предлагаемый'

Save model

In [0]:
!ls

posts_model_words.h5	posts_word_to_int.csv  vocab.json
posts_model_words.json	sample_data


In [0]:
model_json = model.to_json()
json_file = open('posts_model.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model.h5')

In [0]:
!ls

posts_word_to_int.csv  sample_data  vocab.json


In [0]:
from google.colab import files

In [0]:
files.download('posts_model.json')
files.download('posts_model.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 42308, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
files.download('posts_model_words.json')
files .download('posts_model_words.h5')

KeyboardInterrupt: ignored

# Generate by symbols

In [0]:
posts = pd.read_csv('https://raw.githubusercontent.com/Khlebovich-Alexandra/horoscope_generator/master/Data/final_posts.csv', index_col=0)

In [0]:
text = posts.text
text = '\n'.join(text)

print ('Length of text: {} characters'.format(len(text)))

Length of text: 8121558 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проанализировать ситуацию и выяснить, что же так сильно выбило вас из колеи. приниэтого месяца во внимание вашу уязвимость, постарайтесь избегать тех ситуа


In [0]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

с
е
г
о
д


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проана'
'лизировать ситуацию и выяснить, что же так сильно выбило вас из колеи. приниэтого месяца во внимание '
'вашу уязвимость, постарайтесь избегать тех ситуаций, которые требуют проявления всех стальных черт ха'
'рактера. сейчас это для вас будет трудновато.\nвам звёзды рекомендуют сегодня больше работать. ну или '
'хотя бы демонстрировать окружающим имитацию деятельности столь бурной, что у них бы отпало всякое жел'


In [0]:
dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проан'
Target data: 'егодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проана'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 67 ('с')
  expected output: 55 ('е')
Step    1
  input: 55 ('е')
  expected output: 53 ('г')
Step    2
  input: 53 ('г')
  expected output: 64 ('о')
Step    3
  input: 64 ('о')
  expected output: 54 ('д')
Step    4
  input: 54 ('д')
  expected output: 63 ('н')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           22016     
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_5 (Dense)              (64, None, 86)            88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([ 7, 11, 28, 21, 19, 25, 77, 45, 32, 82, 59, 41, 19, 38, 44, 42, 49,
       76, 49, 10, 11, 29,  1, 84, 82, 19, 37,  9, 72, 68, 29, 25, 67, 16,
        5, 68, 65, 65, 37, 73, 10, 27, 65, 79, 77, 21, 27, 66, 25, 81, 41,
       57, 63, 79, 79, 10, 81, 53, 55, 70, 83, 81, 31, 34, 20, 60,  3, 35,
        7, 70, 36, 60, 63, 20, 21, 70, 33, 82, 78, 39, 80, 59,  9, 61, 73,
       52,  2, 52, 43, 85, 69, 34, 19, 76, 20, 28,  1, 73, 11, 15])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'что где-то баланс нарушен, постараться восстановить вопиющую несправедливость, если это, конечно, бу'

Next Char Predictions: 
 '*.a86=ыvhёйr6ous»ъ»-.c “ё6n,цтc=с3(тппnч-_пэы8_р=яrзнээ-ягеф–яek7к#l*фmкн78фiёьpюй,лчв!вt”уk6ъ7a ч.2'


## Train the model

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
history1 = model.fit(dataset, epochs=9)

Train for 1256 steps
Epoch 1/9
1256/1256 [==============================] - 54s 43ms/step - loss: 1.5575
Epoch 2/9
1256/1256 [==============================] - 53s 42ms/step - loss: 1.0875
Epoch 3/9
1256/1256 [==============================] - 53s 42ms/step - loss: 1.0126
Epoch 4/9
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9713
Epoch 5/9
1256/1256 [==============================] - 53s 43ms/step - loss: 0.9456
Epoch 6/9
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9288
Epoch 7/9
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9193
Epoch 8/9
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9143
Epoch 9/9
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9133


In [0]:
model_json = model.to_json()
json_file = open('posts_model_symbols.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model_symbols.h5')

## Generate text

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

model.load_weights('posts_model_symbols.h5')

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_6 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text_symbols(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня рекомендуется стать достойными смелые!
в противном, настоящим другом, и порой появятся возможности для того, чтобы отдохнуть, питание, стрельца и его партнера будет знамен. ваша профессиям и дело прекрасно пониманию вам предназначат и чаще без основной работы, выяснить своим техником скожет вам какое-то используете все свою перспектива и консерватить вас.
завтра вы не так уж весовестность против силпосознить о свой контакт. даже самые валить перспективы. лучше не хотите и водолею не стоит.
в этот период положением ждут проблемысленнего пеле вника, а заниматься тем, кто справились в отношениях с любимым человеком. завтра у льни не будет готовы, полагайтесь – собеседник не ожидается. в этом пожет зависти вас обмануты в этот день не вежливой и женщиной. это не столько думали со своим любимым человеком данный период подход. так есть стороной для него у вас источник, на которых вы давно назрели; и особенно условно происходить: быть могут проявиться самодолго. вот, с конкурентом в об

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня все будет складываться мелкие ошибки бод лишьственнее желание. заниэмом завершить давние общения, недвижимости, от которых можно таким обманутым особым из общения с вашим любимым человеком. планеты решить физические нагрузку, чем ожидая с друзьями. все обещания, изменят. старайтесь ожидать очень внимательны к азартных игр задуманного. вы занятие этому заинтересуется личной имания и больших перемен.
ваши общие интерьера и особого энтузиалось много новых доход и прошлых неожиданных ситуаций.
завтра за холодя праздначастоящим или супругам и слезо действовать. в результате чего вы будете суменичать, чтобы их помощник.
день проша вас могут быть проблемы своего избранника в неизбежным вопросам. но должное время вы рискуете встретить в волшуется вместе с любимым человеком, как можно быстрее вас развиваться. также, поэтому об утренно оформленных вопросов не паников. постарайтесь не попробуйте нежелательно. исключую неожиданное ощущение неизбежно не откладывал, что существует большие пе

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня вам"))

сегодня вам стейнена потока инструмеете, и это может произойти – решать преть чем обманет его и сегодня. в гуще событий к самому знакомству по этому пожелаются безрасплой, велика вероятность травми.
вам лучше заниматься коммерческой деятельностью, хотя пос близких) или заданий, однако вам поселится своцентру максимум восхищении проблем и людей, чтобы разговаривать, как следует действовать дальше.
сегодня ваши временные перспективы от неешеств могут ждать участвовать в вашей задачей долго но отрицательную точку зрения, исполненные последствия относиться ко своегодня уместны легкости, сферы работы. постарайтесь сдерживать свой график к экономическом возрастах.
этот необходимо избавляться хозяйственными делами вазачастьтурения по хозяйственным ответов и итого месяца.
если вы любят очень приятным и одиноко! безготовному отдыху, которое вы будете контролировать какие-то оплошение нового в отношениях с любимым человеком васе, все довольно эмоциональные позиции, разочарования вслух, не исключ

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня ни в коем случае не подходящий день для творчества. удачное время для свиданий для этого воображающие свобода и правильно неотложных впечатлений! кто-то у вас появится хорошо потерять подарок спорах.
ваше настроение они открыта это придется по возможностительными делами и гармонии.
сегодня вас будет затратить смелые идеи могут избегать конфликтов. впрочем, то это и к подуке техники. не требуют приятные и освежи фирмы, неотразимым! проведив родной деятельностью. не поддавайтесь мечте узнать друг друга. занимать круга вечера больше сравнивовать или о воздейятся для отдыха, семьи или деловых партнеров. судьба предсказать стрельцу необходимо будете считаться в том, чтобы занять вам не должно. день амурных дел, переездась в уютном карьере. иначе можно растормошить дела, но вы почувствуете свой день! звезды советуют селевеподходом, и вы сможете постараться, чтобы набраться. наконец, это помощь или коллег, должность – которое давно забудете и дома. звезды запомнитать себя сегодня с ин

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вам будет месяцте, как правался и не спешите расставаться.
завтра у выа милые поездки, прогулка по магазинам вы будете подобны молнии. хорошо, если вы станете заниматься делами, пораньше, нестандартного вображения.
завтра стрельцу стоит рассчитывать на на обычный разговор у друзей, но не откладывайте на весь умей профессиональной формы.
основной почто он дивенно для выа будет играционально менять традиционные события. вас ожидает обычные известия должны, и особенно осуществлять кредиты или сферы своей страбования, чтобы справиться с ярким промним! ающим компутии лучше отложить, но нельзя мелким заняться переоценки. семейных представителей в будущем очень леустоящим к спунспредставлениям, какое дорогожество. много коротких, удовольствий, это поможет вам, если вы встретится благополучно в целом этот день сделает удовольствия.
завтра вы и ее такая жизнь интересов и конфликтов. ваш октучшей внимания, или. не торопитесь ссловиться от судьбой. постарайтесь меньше переносить со всеми 

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня только не приниэтот месяцте сегодняшним уютном месте. возможны вещие сны в отношениях с опнамными испытывать правильномыпулят, пикантный контроль воспитанием, но написания ко всему, в этот день вым партнеру будет слышного перерасти в росту вашей жизни, иначе рисковать. как бовтакт, попытающественно изоизвит. или же вы успеете с безразличием в контактов, короткие победы, использовав прошлом вы можете блестящих избранником нуждающихся. это идеалена отношения с человеком, которого он хочет лежить новым углом. вам необходимо либо проже его на интересы пройдет понять вашему любимому человеку. возможно, с день гарантижных или со своим любимым человеком и остальными делами. можете плоить день в коллективе.
ваш мств надежный конфликт.
работа пройдет к самому началу сюрпризов, насмешким решением встречи с друзьями. друзья и мероприятия могут очень напряженный день. от вас не вовремению.
серьезно вься за свою линию по отношению к своим будущим. кроме того, завтра  вы постарат вас завязат

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня она я заодности любят стрельца к тому, какие несчастье тратит и беспокоиться, накоплены друг к другу, которой вы будете использовать для дружеского общения. возможно, что в этот день весьма обидеть его в том же духе.
вероятно, фантазии – досконеенность в долгий ящик. постарайтесь проявить внимательное решение в лучшем видения. не доверяйтесь ни стремьесного желания или не вобластить в отношениях с любимым человеком. ну абсолютно бессмысленно наверстать на провокации и открыто нарядное участие.
сегодняшний день напрятьте редко и глубоко. постарайтесь отвечать из неплохо быть в конфликт. главное, чем долго незнакомыми, точностью рассудков и займитесь, без этого строить голову, ожидает полный страстей, способных надежды окончательно выражены скоро людей, которое может иметь сюрпризы: он находятся дополнительные разнообразие ваши жизненные силы. противоречивый день, но не стоит заниматься физической или спонтанной, нет меньшеешный взгляд – одня у стрельца имеют буче человеку, вы ри

In [0]:
print(generate_text_symbols(model, start_string=u"вам "))

вам необходимо великолепным перед старомолять сплете не будет сильнее.
вас не удастся: «для новых напрстремы решатся, и вы сможете найти ответы на повтерника). возможно, на ключок вами поймут: во всем, что окажется прежния. готовность к менедним или сюрпризом. не фтичивайте на себя неприятную и личную жизнь. это мешает потребовляться, и опытом ширтра делая копли еланев и средств.
сегодня утром вы сегодня могут, появится небезные планы. есть опасность нервозности, то избегайте перемен, вы сможете выполнить в дороже и плечи. звезды говоря вы оказались в поездку в каком-то колебание профессиональной деятельности. чужой точки зренного друга.
сегодносительному эмоциональном понисте, откажитесь от установления конструктов. звезды рекомендуют проявить дисциплина, а зочеут с мелкими непростой работой.
день благоприятный для новых знакомств и искренне.
многое зависит от успехи какое-то вра помощь.
этот день обещает принесет хорошие идеи открываются потребление обязанности. вы будете заниматься 

In [0]:
print(generate_text_symbols(model, start_string=u"вас "))

вас могут оказаться потому, чтобы узнать ответную ревновать. легкое очень удивитетельный, но не будьте особенно энтузиазм. наорой деятельностью. ваше желание, если резонтальное тайну с позитивным тончить в кулак завтра . с единомышленникам бе высокого сотрудерозировали в отношениях между «кардинали». во второй половине его нарушение, даже такое вдохновение не принесет масштабных проблем.
вы даже небольшая творческого ужина, однако не стоит браться. деятельность, а потом уже завтра сделать, чтобы «не воспользуются. это поможет намеченную у стадиен, нынче к вам невурным и «или добиваясьмым и приятием выа. ваша жирофальства, но вряд ли могут нарушить энергение окружающих длительного беспричинного планы тропытника.
ваша виновна ситуации. многие выпылка сильна будут воплоть перед собой финансовые потерять кого-либо в творчестве. у вас есть возможность посмотреть по щаксимум выгоды.
вдохновение вещей поможет ему стоит: это много заработка, но, сегодня людей – как бы вам изменить. день подход

In [0]:
print(generate_text_symbols(model, start_string=u"вы "))

вы к работе - любимый человек пройдет ощущаться продвинуться в подарки сегодня удастся – то есть шанс, что все то необходимо избавиться от всяких решений.
завтра день пройденный интерес к окружающим в отношения с любимым человеком. не бойтесь, это только любви одна сфере разговорах начнете добиться. много значительной мозгадец, действуяс касается комплименто о дня внимания ока, потерпит известие. но не развивать мелкие проблемы. вас восприметя активной способности на любимого человека, в которых вы думал, вы рискуете урвать, а заодно и интерес через уровне.
не бвия и конфликта с душу центрального отношения. ограничен, тем неожиданное время для индивидся с интересами близких людей, которые станут жадкой. обратите круг общественных контактов, дольше. оказавшисься на большей работе и конфликтам. купли-принялись по возможностью дополните. ваша энергия относится к этому техную на весь вся за культирествует противоречивого интересов.
в этот день то, что вы ее.
этим воогщании сегодняшнего дня

In [0]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня повышенная усация новый вателор этом подходящий.
не переезжать близких людей. любое демонстрируя информация, которые вряд ли помогли. в романтические отношения споять на месяцсь, опытно, если это вы рияете проблему. вам понадобится в их глаз
какие-то опрочестволжность наслаждаться безрезультатным исключены собой и получить заметную финансовую поддержку.
завтра  устроить жизненный потенциал, без внимания одну из дивиаего и следует особенно трудно продвинуться во взаимоспокояния. на самом делю воображению вашества или совместное мероприятие. нового знаком постарайтесь раскребованы.
вы накопите немало странного педанной жизни м личных и мечты.
возможны споры во всем и действенным нагругом в своих лечениях нибудь о том, что не стоит вести, но и словами.
завтра, есть все электронной напряженности, напротив, партнеров он не должна требывать вескидете или останутся людьми, которые вызывают действия со своим избранником. это может привести к финансовым вопрос получения не в ладом следу

In [0]:
model_json = model.to_json()
json_file = open('posts_model.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model.h5')

In [0]:
files.download('posts_model.json')
files.download('posts_model.h5')

In [0]:
files.download('posts_model_symbols.json')
files.download('posts_model_symbols.h5')